In [1]:
!pip install sounddevice
!pip install numpy
!pip install tkinter


Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for sounddevice from https://files.pythonhosted.org/packages/bf/2a/58fa1704b5cf8041564337674790426d39d630e7407e54e17a1212332959/sounddevice-0.5.0-py3-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/189.8 kB ? eta -:--:--
   -- ------------------------------------- 10.2/189.8 kB ? eta -:--:--
   ------ -------------------------------- 30.7/189.8 kB 325.1 kB/s eta 0:00:01
   ------------------------- -------------- 122.9/189.8 kB 1.0 MB/s eta 0:00:01
   -------------------------------------- - 184.3/189.8 kB 1.4 MB/s eta 0:00:01
   ---------------------------------------- 189.8/189.8 kB 1.0 MB/s eta 0:00:00
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement tkinter (from versions: none)
ERROR: No matching distribution found for tkinter


In [ ]:
import sounddevice as sd
import numpy as np
import wave
import tkinter as tk
from tkinter import filedialog, messagebox
import threading

# Global variables for recording
fs = 44100  # Sample rate (44.1kHz is standard)
recording = False
frames = []

# Function to start recording
def start_recording():
    global recording, frames
    recording = True
    frames = []
    # Create a thread for recording to prevent blocking the GUI
    threading.Thread(target=record_audio).start()

# Function to record audio
def record_audio():
    global frames
    with sd.InputStream(samplerate=fs, channels=2, callback=audio_callback):
        while recording:
            sd.sleep(100)

# Callback function to store audio data
def audio_callback(indata, frames_count, time, status):
    global frames
    frames.append(indata.copy())

# Function to stop recording
def stop_recording():
    global recording
    recording = False

# Function to save recording
def save_recording():
    global frames
    if len(frames) == 0:
        messagebox.showinfo("Info", "No recording to save!")
        return
    
    # Open file dialog to save the file
    filepath = filedialog.asksaveasfilename(defaultextension=".wav", filetypes=[("WAV files", "*.wav")])
    if filepath:
        # Save the recording to a WAV file
        with wave.open(filepath, 'wb') as wf:
            wf.setnchannels(2)
            wf.setsampwidth(2)
            wf.setframerate(fs)
            wf.writeframes(b''.join(np.array(frames, dtype=np.int16).tobytes()))
        messagebox.showinfo("Info", f"Recording saved as {filepath}")

# Create a simple UI using tkinter
def create_ui():
    root = tk.Tk()
    root.title("Audio Recorder")
    
    start_button = tk.Button(root, text="Start Recording", command=start_recording)
    start_button.pack(pady=10)
    
    stop_button = tk.Button(root, text="Stop Recording", command=stop_recording)
    stop_button.pack(pady=10)
    
    save_button = tk.Button(root, text="Save Recording", command=save_recording)
    save_button.pack(pady=10)
    
    root.mainloop()

# Run the UI
create_ui()
